### 记录数据预处理后

从原始数据的mhd数据进行ROI处理（获取肺部ROI），并统一分辨率。

经过这样的操作，我们得到的每个病例的大小是不同的，

In [ ]:
def check_nan(path):
    """
    检查当前npy文件中是否有nan值
    """
    
    for file in os.listdir(path):
        arr = np.load(os.path.join(path,file))
        
        a = arr[np.isnan(arr)]
        if (len(a)>0):
            print("file is nan": file)
            print(a)

In [ ]:
def plot_nodule(npy_file):
    """
    绘制结节，一张一张绘制
    """
    nodule_arr = np.load(npy_file)
    f, plots = plt.subplots(int(nodule_arr.shape[0]/3), 3, figsize=(50,50))
    
    for i in range(0,nodule_arr.shape[0]):
        plots[int(i/3), int((i % 3))].axis("off")
        plots[int(i/3), int((i % 3))].imshow(nodule_arr[i,:,:], cmap=plt.cm.bone)

In [ ]:
def search(path, word):
    """
    从路径查找文件名匹配关键字
    path: 要查找的路径
    word：要匹配的关键字
    """
    filelist = []
    
    for filename in os.listdir(path):
        fp = os.path.join(path, filename)
        if os.path.isfile(fp) and word in filename:
            filelist.append(fp)
        elif os.path.isdir(fp):
            serch(fp, word)
    return filelist

In [ ]:
def angle_transpose(file, degree, flag_string):
    """
    图像变换：旋转图像
    file：一个npy文件
    degree: 图像旋转度数，90，180，270可以
    flag_string：在转置后将flag_string标记添加到文件名中
    
    90: leftright
    180: updown
    270: diagonal
    """
    array = np.load(file)  # z,y,x
    newarr = np.zeros(array.shape,dtype=uint8)
    for depth in range(array.shape[0]):
        slice_arr = array[depth]
        slice_arr.reshape((slice_arr.shape[0],slice_arr.shape[1], 1))
        img = Image.fromarray(slice_arr)
        
        out = img.rotate(degree) # 逆时针旋转degree度
        
        newarr[depth,:,:] = np.array(out).reshape(array.shape[1],-1)[:,:]
    np.save(file.replace(".npy",flag_string+".npy"), newarr)

In [ ]:
# 对肺部mask求取边界，只保留边界内的数据，不妨想象一个正方形，
# 里面有一个小圆，小圆就是掩码，那么此处做的就是求小圆的最小外接平行矩形，将矩形外的部分砍掉
# 这个矩形是与图像平行的。

# 以下为主要代码
# Mask的形状为(D,H,W)<=>(z,y,x)
# sliceim1的形状为(D,H,W)<=>(z,y,x)

zz, yy, xx = np.where(Mask)
print(zz,yy,xx)
box = np.array([[np.min(zz),np.max(zz)],[np.min(yy),np.max(yy)], [np.min(xx),np.max(xx)]])
print(box)
box = np.floor(box).astype('int') # 保证box为整数
print(box)


# 对这个边界向外扩展一点，为了处理边缘的像素,左边坐标减5，右边坐标加10。
margin = 5
extendbox = np.vstack([np.max([[0,0,0],box[:,0]-margin],0),np.min([newshape,box[:,1]+2*margin],axis=0).T]).T
print(extendbox)

# 将extendbox内数据取出作为最后结果 
sliceim2 = sliceim1[extendbox[0,0]:extendbox[0,1],
                    extendbox[1,0]:extendbox[1,1],
                    extendbox[2,0]:extendbox[2,1]]